<center style><h3>Computer Assignment 2</h3></center>
<center style><h4>Game - Othello</h4></center>
 <h4 style="text-align:right">
        محمد پویا افشاری - 810198577
</h4>

### Part 1 : simply writing a get_human_move

Heuristic:
get_human_moves, heuristic choose the best move for the human player. The heuristic is to choose the move that captures the most opponent pieces. The code iterates through all the possible moves that the human player can make, and for each move, it counts the number of opponent pieces that can be captured by that move. The move that captures the most opponent pieces is considered the best move and returned.

In [84]:
import random
import time
import turtle


class OthelloUI:
    def __init__(self, board_size=6, square_size=60):
        self.board_size = board_size
        self.square_size = square_size
        self.screen = turtle.Screen()
        self.screen.setup(self.board_size * self.square_size + 50, self.board_size * self.square_size + 50)
        self.screen.bgcolor('white')
        self.screen.title('Othello')
        self.pen = turtle.Turtle()
        self.pen.hideturtle()
        self.pen.speed(0)
        turtle.tracer(0, 0)

    def draw_board(self, board):
        self.pen.penup()
        x, y = -self.board_size / 2 * self.square_size, self.board_size / 2 * self.square_size
        for i in range(self.board_size):
            self.pen.penup()
            for j in range(self.board_size):
                self.pen.goto(x + j * self.square_size, y - i * self.square_size)
                self.pen.pendown()
                self.pen.fillcolor('green')
                self.pen.begin_fill()
                self.pen.setheading(0)
                for _ in range(4):
                    self.pen.forward(self.square_size)
                    self.pen.right(90)
                self.pen.penup()
                self.pen.end_fill()
                self.pen.goto(x + j * self.square_size + self.square_size / 2,
                              y - i * self.square_size - self.square_size + 5)
                if board[i][j] == 1:
                    self.pen.fillcolor('white')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()
                elif board[i][j] == -1:
                    self.pen.fillcolor('black')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()

        turtle.update()


class Othello:
    def __init__(self, ui, minimax_depth=1, prune=True):
        self.size = 6
        self.ui = OthelloUI(self.size) if ui else None
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.board[int(self.size / 2) - 1][int(self.size / 2) - 1] = self.board[int(self.size / 2)][
            int(self.size / 2)] = 1
        self.board[int(self.size / 2) - 1][int(self.size / 2)] = self.board[int(self.size / 2)][
            int(self.size / 2) - 1] = -1
        self.current_turn = random.choice([1, -1])
        self.minimax_depth = minimax_depth
        self.prune = prune

    def get_winner(self):
        white_count = sum([row.count(1) for row in self.board])
        black_count = sum([row.count(-1) for row in self.board])
        if white_count > black_count:
            return 1
        elif white_count < black_count:
            return -1
        else:
            return 0

    def get_valid_moves(self, player):
        moves = set()
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            x, y = i, j
                            captured = []
                            while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == -player:
                                captured.append((x + di, y + dj))
                                x += di
                                y += dj
                            if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == player and len(captured) > 0:
                                moves.add((i, j))
        return list(moves)

    def make_move(self, player, move):
        i, j = move
        self.board[i][j] = player
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                x, y = i, j
                captured = []
                while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == -player:
                    captured.append((x + di, y + dj))
                    x += di
                    y += dj
                if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == player:
                    for (cx, cy) in captured:
                        self.board[cx][cy] = player

    def get_cpu_move(self):
        moves = self.get_valid_moves(-1)
        if len(moves) == 0:
            return None
        return random.choice(moves)
    
    def get_human_move(self):
        moves = self.get_valid_moves(1)
        if len(moves) == 0:
            return None
        
        # Heuristic: choose the move that captures the most opponent pieces.
        best_move = None
        max_captured = 0
        for move in moves:
            i, j = move
            captured = 0
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    if di == 0 and dj == 0:
                        continue
                    x, y = i, j
                    while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == -1:
                        x += di
                        y += dj
                    if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == 1:
                        captured += abs(x - i) + abs(y - j)
            if captured > max_captured:
                best_move = move
                max_captured = captured
        
        return best_move



    
    def minimax(self, player, opponent, depth, alpha=float('-inf'), beta=float('inf')):
        if depth == 0 or self.terminal_test():
            return self.evaluate(), None
    
        best_value = float('-inf') if player == -1 else float('inf')
        best_move = None
    
        for move in self.get_valid_moves(player):
            self.make_move(player, move)
            value, _ = self.minimax(opponent, player, depth - 1, alpha, beta)
            self.make_move(0, move)
    
            if player == -1 and value < best_value:
                best_value = value
                best_move = move
                beta = min(beta, best_value)
                if self.prune and beta <= alpha:
                    break
            elif player == 1 and value > best_value:
                best_value = value
                best_move = move
                alpha = max(alpha, best_value)
                if self.prune and beta <= alpha:
                    break
                
        return best_value, best_move
    
    def evaluate(self):
        # Heuristic: evaluate the difference in the number of pieces between the players.
        white_count = sum([row.count(1) for row in self.board])
        black_count = sum([row.count(-1) for row in self.board])
        return white_count - black_count


    def terminal_test(self):
        return len(self.get_valid_moves(1)) == 0 and len(self.get_valid_moves(-1)) == 0

    def play(self):
        winner = None
        while not self.terminal_test():
            if self.ui:
                self.ui.draw_board(self.board)
            if self.current_turn == 1:
                move = self.get_human_move()
                if move:
                    self.make_move(self.current_turn, move)
            else:
                move = self.get_cpu_move()
                if move:
                    self.make_move(self.current_turn, move)
            self.current_turn = -self.current_turn
            if self.ui:
                self.ui.draw_board(self.board)
                time.sleep(1)
        winner = self.get_winner()
        return winner


هدف تصمیم گیری حرکات یک بازیکن در بازی Othello به کمک الگوریتم Minmax


The game is played between a human player and a computer player. The computer player uses a simple random algorithm to make its moves.

The Othello class contains the logic for the game itself. The get_valid_moves() method returns a list of valid moves that a player can make, while the make_move() method applies a move to the board. The play() method is the main game loop that alternates between the human and computer players until the game is over.

The OthelloUI class handles the user interface. The draw_board() method draws the board using Turtle, and the update() method updates the screen.

In [85]:
#game = Othello(ui = False,minimax_depth=4,prune=False)
#game.play()

* We first set the number of games to play (num_games) to 150, and the depth of the minimax algorithm (depth) to 1,3, and 5.

* We then initialize variables to keep track of the total time taken to play all the games (total_time) and the number of wins (num_wins).

* We use a loop to play num_games games. For each game, we create a new Othello object with the specified depth and ui=False to disable the user interface.

* We use the time module to record the start and end time of each game and compute the total time taken to play all the games.

* After each game, we check the winner of the game (winner). If winner is 1, it means that the computer player won, so we increment num_wins.

* Finally, we compute the average time per game by dividing total_time by num_games, and print out the results.

In [86]:
import time

num_games = 150
depth = 1
total_time = 0
num_wins = 0

for i in range(num_games):
    game = Othello(ui=False, minimax_depth=depth, prune=False)
    start_time = time.time()
    winner = game.play()
    end_time = time.time()
    total_time += (end_time - start_time)
    if winner == 1:
        num_wins += 1

avg_time = total_time / num_games
print("Average time per game:", avg_time)
print("Number of wins:", num_wins)


Average time per game: 0.0049230162302652996
Number of wins: 98


In [87]:
import time

num_games = 150
depth = 3
total_time = 0
num_wins = 0

for i in range(num_games):
    game = Othello(ui=False, minimax_depth=depth, prune=False)
    start_time = time.time()
    winner = game.play()
    end_time = time.time()
    total_time += (end_time - start_time)
    if winner == 1:
        num_wins += 1

avg_time = total_time / num_games
print("Average time per game:", avg_time)
print("Number of wins:", num_wins)


Average time per game: 0.004523905118306478
Number of wins: 106


In [88]:
import time

num_games = 150
depth = 5
total_time = 0
num_wins = 0

for i in range(num_games):
    game = Othello(ui=False, minimax_depth=depth, prune=False)
    start_time = time.time()
    winner = game.play()
    end_time = time.time()
    total_time += (end_time - start_time)
    if winner == 1:
        num_wins += 1

avg_time = total_time / num_games
print("Average time per game:", avg_time)
print("Number of wins:", num_wins)


Average time per game: 0.0045107634862263995
Number of wins: 103


### Part 2 : Add minimax to the code

The heuristic used in the code is a simple piece-counting evaluation function, where the value of a board is determined by the difference between the number of white pieces and the number of black pieces on the board. This is implemented in the evaluate function, which sums the number of 1s (representing white pieces) and subtracts the number of -1s (representing black pieces) in the board matrix.

In [89]:
import random
import time
import turtle


class OthelloUI:
    def __init__(self, board_size=6, square_size=60):
        self.board_size = board_size
        self.square_size = square_size
        self.screen = turtle.Screen()
        self.screen.setup(self.board_size * self.square_size + 50, self.board_size * self.square_size + 50)
        self.screen.bgcolor('white')
        self.screen.title('Othello')
        self.pen = turtle.Turtle()
        self.pen.hideturtle()
        self.pen.speed(0)
        turtle.tracer(0, 0)

    def draw_board(self, board):
        self.pen.penup()
        x, y = -self.board_size / 2 * self.square_size, self.board_size / 2 * self.square_size
        for i in range(self.board_size):
            self.pen.penup()
            for j in range(self.board_size):
                self.pen.goto(x + j * self.square_size, y - i * self.square_size)
                self.pen.pendown()
                self.pen.fillcolor('green')
                self.pen.begin_fill()
                self.pen.setheading(0)
                for _ in range(4):
                    self.pen.forward(self.square_size)
                    self.pen.right(90)
                self.pen.penup()
                self.pen.end_fill()
                self.pen.goto(x + j * self.square_size + self.square_size / 2,
                              y - i * self.square_size - self.square_size + 5)
                if board[i][j] == 1:
                    self.pen.fillcolor('white')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()
                elif board[i][j] == -1:
                    self.pen.fillcolor('black')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()

        turtle.update()


class Othello:
    def __init__(self, ui, minimax_depth=1, prune=True):
        self.size = 6
        self.ui = OthelloUI(self.size) if ui else None
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.board[int(self.size / 2) - 1][int(self.size / 2) - 1] = self.board[int(self.size / 2)][
            int(self.size / 2)] = 1
        self.board[int(self.size / 2) - 1][int(self.size / 2)] = self.board[int(self.size / 2)][
            int(self.size / 2) - 1] = -1
        self.current_turn = random.choice([1, -1])
        self.minimax_depth = minimax_depth
        self.prune = prune

    def get_winner(self):
        white_count = sum([row.count(1) for row in self.board])
        black_count = sum([row.count(-1) for row in self.board])
        if white_count > black_count:
            return 1
        elif white_count < black_count:
            return -1
        else:
            return 0

    def get_valid_moves(self, player):
        moves = set()
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            x, y = i, j
                            captured = []
                            while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == -player:
                                captured.append((x + di, y + dj))
                                x += di
                                y += dj
                            if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == player and len(captured) > 0:
                                moves.add((i, j))
        return list(moves)

    def make_move(self, player, move):
        i, j = move
        self.board[i][j] = player
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                x, y = i, j
                captured = []
                while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == -player:
                    captured.append((x + di, y + dj))
                    x += di
                    y += dj
                if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == player:
                    for (cx, cy) in captured:
                        self.board[cx][cy] = player

    def get_cpu_move(self):
        moves = self.get_valid_moves(-1)
        if len(moves) == 0:
            return None
        return random.choice(moves)

    def get_human_move(self):
        moves = self.get_valid_moves(1)
        if len(moves) == 0:
            return None
        best_move = None
        max_val = float('-inf')
        for move in moves:
            new_board = [row[:] for row in self.board]
            self.make_move(1, move)
            val = self.minimax(new_board, self.minimax_depth, False)
            if val > max_val:
                max_val = val
                best_move = move
            self.board = new_board
        return best_move

    def minimax(self, board, depth, maximizing_player):
        if depth == 0 or self.terminal_test():
            return self.evaluate(board)
        if maximizing_player:
            max_val = float('-inf')
            moves = self.get_valid_moves(-1)
            for move in moves:
                new_board = [row[:] for row in board]
                self.make_move(-1, move)
                val = self.minimax(new_board, depth - 1, False)
                max_val = max(max_val, val)
                self.board = board
            return max_val
        else:
            min_val = float('inf')
            moves = self.get_valid_moves(1)
            for move in moves:
                new_board = [row[:] for row in board]
                self.make_move(1, move)
                val = self.minimax(new_board, depth - 1, True)
                min_val = min(min_val, val)
                self.board = board
            return min_val

    def evaluate(self, board):
        return sum([row.count(1) for row in board]) - sum([row.count(-1) for row in board])


    def terminal_test(self):
        return len(self.get_valid_moves(1)) == 0 and len(self.get_valid_moves(-1)) == 0

    def play(self):
        winner = None
        while not self.terminal_test():
            if self.ui:
                self.ui.draw_board(self.board)
            if self.current_turn == 1:
                move = self.get_human_move()
                if move:
                    self.make_move(self.current_turn, move)
            else:
                move = self.get_cpu_move()
                if move:
                    self.make_move(self.current_turn, move)
            self.current_turn = -self.current_turn
            if self.ui:
                self.ui.draw_board(self.board)
                time.sleep(1)
        winner = self.get_winner()
        return winner


In [90]:
import time

num_games = 150
depth = 1
total_time = 0
num_wins = 0

for i in range(num_games):
    game = Othello(ui=False, minimax_depth=depth, prune=False)
    start_time = time.time()
    winner = game.play()
    end_time = time.time()
    total_time += (end_time - start_time)
    if winner == 1:
        num_wins += 1

avg_time = total_time / num_games
print("Average time per game:", avg_time)
print("Number of wins:", num_wins)


Average time per game: 0.001280350685119629
Number of wins: 150


In [91]:
import time

num_games = 150
depth = 3
total_time = 0
num_wins = 0

for i in range(num_games):
    game = Othello(ui=False, minimax_depth=depth, prune=False)
    start_time = time.time()
    winner = game.play()
    end_time = time.time()
    total_time += (end_time - start_time)
    if winner == 1:
        num_wins += 1

avg_time = total_time / num_games
print("Average time per game:", avg_time)
print("Number of wins:", num_wins)


Average time per game: 0.013571909268697103
Number of wins: 122


In [92]:
import time

num_games = 150
depth = 5
total_time = 0
num_wins = 0

for i in range(num_games):
    game = Othello(ui=False, minimax_depth=depth, prune=False)
    start_time = time.time()
    winner = game.play()
    end_time = time.time()
    total_time += (end_time - start_time)
    if winner == 1:
        num_wins += 1

avg_time = total_time / num_games
print("Average time per game:", avg_time)
print("Number of wins:", num_wins)


Average time per game: 0.10308587551116943
Number of wins: 150


### Add alpha-beta pruning

I have added two extra parameters to the minimax function, named 'alpha' and 'beta', which represent the lower and upper bounds of the best value found so far for the maximizing player and the minimizing player, respectively. In each recursive call to the function, we update these values and check if we can prune the search by comparing them. Specifically, if beta becomes less than or equal to alpha, we break out of the loop and return the best value found so far.

In [93]:
import random
import time
import turtle


class OthelloUI:
    def __init__(self, board_size=6, square_size=60):
        self.board_size = board_size
        self.square_size = square_size
        self.screen = turtle.Screen()
        self.screen.setup(self.board_size * self.square_size + 50, self.board_size * self.square_size + 50)
        self.screen.bgcolor('white')
        self.screen.title('Othello')
        self.pen = turtle.Turtle()
        self.pen.hideturtle()
        self.pen.speed(0)
        turtle.tracer(0, 0)

    def draw_board(self, board):
        self.pen.penup()
        x, y = -self.board_size / 2 * self.square_size, self.board_size / 2 * self.square_size
        for i in range(self.board_size):
            self.pen.penup()
            for j in range(self.board_size):
                self.pen.goto(x + j * self.square_size, y - i * self.square_size)
                self.pen.pendown()
                self.pen.fillcolor('green')
                self.pen.begin_fill()
                self.pen.setheading(0)
                for _ in range(4):
                    self.pen.forward(self.square_size)
                    self.pen.right(90)
                self.pen.penup()
                self.pen.end_fill()
                self.pen.goto(x + j * self.square_size + self.square_size / 2,
                              y - i * self.square_size - self.square_size + 5)
                if board[i][j] == 1:
                    self.pen.fillcolor('white')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()
                elif board[i][j] == -1:
                    self.pen.fillcolor('black')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()

        turtle.update()


class Othello:
    def __init__(self, ui, minimax_depth=1, prune=True):
        self.size = 6
        self.ui = OthelloUI(self.size) if ui else None
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.board[int(self.size / 2) - 1][int(self.size / 2) - 1] = self.board[int(self.size / 2)][
            int(self.size / 2)] = 1
        self.board[int(self.size / 2) - 1][int(self.size / 2)] = self.board[int(self.size / 2)][
            int(self.size / 2) - 1] = -1
        self.current_turn = random.choice([1, -1])
        self.minimax_depth = minimax_depth
        self.prune = prune

    def get_winner(self):
        white_count = sum([row.count(1) for row in self.board])
        black_count = sum([row.count(-1) for row in self.board])
        if white_count > black_count:
            return 1
        elif white_count < black_count:
            return -1
        else:
            return 0

    def get_valid_moves(self, player):
        moves = set()
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            x, y = i, j
                            captured = []
                            while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == -player:
                                captured.append((x + di, y + dj))
                                x += di
                                y += dj
                            if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == player and len(captured) > 0:
                                moves.add((i, j))
        return list(moves)

    def make_move(self, player, move):
        i, j = move
        self.board[i][j] = player
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                x, y = i, j
                captured = []
                while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == -player:
                    captured.append((x + di, y + dj))
                    x += di
                    y += dj
                if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == player:
                    for (cx, cy) in captured:
                        self.board[cx][cy] = player

    def get_cpu_move(self):
        moves = self.get_valid_moves(-1)
        if len(moves) == 0:
            return None
        return random.choice(moves)
    
    def get_human_move(self):
        moves = self.get_valid_moves(1)
        if len(moves) == 0:
            return None
        best_move = None
        max_val = float('-inf')
        for move in moves:
            new_board = [row[:] for row in self.board]
            self.make_move(1, move)
            val = self.minimax(new_board, self.minimax_depth, float('-inf'), float('inf'), False)
            if val > max_val:
                max_val = val
                best_move = move
            self.board = new_board
        return best_move

    def minimax(self, board, depth, alpha, beta, maximizing_player):
        if depth == 0 or self.terminal_test():
            return self.evaluate(board)
        if maximizing_player:
            max_val = float('-inf')
            moves = self.get_valid_moves(-1)
            for move in moves:
                new_board = [row[:] for row in board]
                self.make_move(-1, move)
                val = self.minimax(new_board, depth - 1, alpha, beta, False)
                max_val = max(max_val, val)
                self.board = board
                alpha = max(alpha, val)
                if beta <= alpha:
                    break
            return max_val
        else:
            min_val = float('inf')
            moves = self.get_valid_moves(1)
            for move in moves:
                new_board = [row[:] for row in board]
                self.make_move(1, move)
                val = self.minimax(new_board, depth - 1, alpha, beta, True)
                min_val = min(min_val, val)
                self.board = board
                beta = min(beta, val)
                if beta <= alpha:
                    break
            return min_val

    def evaluate(self, board):
        return sum([row.count(1) for row in board]) - sum([row.count(-1) for row in board])


    def terminal_test(self):
        return len(self.get_valid_moves(1)) == 0 and len(self.get_valid_moves(-1)) == 0

    def play(self):
        winner = None
        while not self.terminal_test():
            if self.ui:
                self.ui.draw_board(self.board)
            if self.current_turn == 1:
                move = self.get_human_move()
                if move:
                    self.make_move(self.current_turn, move)
            else:
                move = self.get_cpu_move()
                if move:
                    self.make_move(self.current_turn, move)
            self.current_turn = -self.current_turn
            if self.ui:
                self.ui.draw_board(self.board)
                time.sleep(1)
        winner = self.get_winner()
        return winner


In [94]:
import time

num_games = 150
depth = 5
total_time = 0
num_wins = 0

for i in range(num_games):
    game = Othello(ui=False, minimax_depth=depth, prune=True)
    start_time = time.time()
    winner = game.play()
    end_time = time.time()
    total_time += (end_time - start_time)
    if winner == 1:
        num_wins += 1

avg_time = total_time / num_games
print("Average time per game:", avg_time)
print("Number of wins:", num_wins)


Average time per game: 0.02261721611022949
Number of wins: 150


In [95]:
import time

num_games = 150
depth = 7
total_time = 0
num_wins = 0

for i in range(num_games):
    game = Othello(ui=False, minimax_depth=depth, prune=True)
    start_time = time.time()
    winner = game.play()
    end_time = time.time()
    total_time += (end_time - start_time)
    if winner == 1:
        num_wins += 1

avg_time = total_time / num_games
print("Average time per game:", avg_time)
print("Number of wins:", num_wins)


Average time per game: 0.15435375213623048
Number of wins: 146


### Questions:

1. What things did you consider when calculating your heuristic and why?

The heuristic used in `evaluate` method is a simple differeence between number of player and CPu players pieces on the board in each state. The reason is that it provides a quick way to evaluate the board state without having to perform an exhaustive search of all possible future states.

The heuristic is based on the assumption that having more pieces on the board than the opponent is generally a good thing, and this assumption holds true for Othello. Therefore, In part 1 to make human moves I have considered `best_move` the move which flips most pieces of the opponent.

But these heuristics are not perfect because it doesnt consider any strategic place of the pieces on the board and can be improved.


2. Do you see any relationship between the depth of the algorithm and the calculated parameters? Check thoroughly how the depth of the algorithm affects the chance of winning, time and nodes seen.

Yes, there is a relationship between the depth of the algorithm and the calculated parameters. Increasing the depth of the algorithm will generally lead to better chances of winning, but at the cost of increased time and nodes seen. This is because as the depth increases, the algorithm considers more future moves and their potential outcomes, leading to a more accurate evaluation of the game state. However, this also means that the algorithm has to search through more possible moves and game states, which takes more time and resources.

3. Can we choose the order of seeing the children of each node in such a way that we have the most pruning? If your answer is positive, explain your method, and if not, explain why this action is not possible.

Yes, we can choose the order of seeing the children of each node in such a way that we have the most pruning. For the method to do so, we might prioritize moves that capture opponent pieces or create opportunities for future captures. By ordering the children in this way, we can potentially prune more subtrees because we are more likely to find good moves earlier in the search.

4. Explain Factor Branching and say what changes it will make with the progress of this game?

Factor Branching is a technique used to reduce the size of the game tree by selectively expanding only a subset of the children of each node. In this technique, the algorithm evaluates each child node based on a certain factor, such as the probability of winning or the estimated value of the node. Then, the algorithm only expands the most promising nodes, while pruning the others. This can significantly reduce the size of the game tree, leading to faster and more efficient algorithms.

5. Explain why the algorithm becomes faster without losing accuracy during pruning.

The algorithm keeps accuracy and becomes faster becauses it eliminates unnecessary branches from the game tree. By eliminating these branches, the algorithm can reduce the number of nodes it needs to evaluate, which saves time and resources. The accuracy is not affected because the algorithm only prunes branches that are guaranteed to lead to worse outcomes, while keeping the most promising branches intact.

6. Why is using minimax not the most optimal method in situations where the opponent acts by chance (such as this project)? What algorithm can replace this algorithm? Explain completely.

In the situations opponents acts randomly minimax algorithm is not an oprimal solution, because it consider the opponent choose the best move it can make and wort move for player, which is not always true in the situations where opponent acts randomly. In such a situations, we may consider a move carefully and stay in the safe-zone but if we try other paths, the opponent may give us a bunus point which resualts us to win with a higher point.

In such situations, a more appropriate algorithm is Monte Carlo Tree Search (MCTS). MCTS is a heuristic search algorithm that randomly simulates gameplay to estimate the value of each move, and then selects the move with the highest estimated value. By simulating many random games, MCTS can gradually build a better estimate of the value of each move, even when the opponent is acting randomly. This makes MCTS a more appropriate algorithm in situations where the opponent is acting by chance.